In [5]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier 

In [6]:
train_data = pd.read_csv("data_train.csv")

In [7]:
train_data.head()

,Id,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEPARTING_AIRPORT,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,DEP_TIME_BLK,DISTANCE,...,GROUND_SERV_PER_PASS,PLANE_AGE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND,DEP_DELAY_NEW,IS_DELAYED
0,0,1,17,4,Raleigh-Durham International,"Raleigh/Durham, NC",JFK,"New York, NY",1800-1859,427,...,0.000094,5,Ronald Reagan Washington National,0.01,0.0,0.0,49.0,4.70,2.0,1
1,1,2,27,3,San Jose International,"San Jose, CA",BOS,"Boston, MA",2200-2259,2689,...,0.000127,17,John F. Kennedy International,0.21,0.0,0.0,64.0,12.75,0.0,0
2,2,8,14,3,Los Angeles International,"Los Angeles, CA",ATL,"Atlanta, GA",0600-0659,1947,...,0.000149,23,NONE,0.00,0.0,0.0,74.0,8.50,0.0,0
3,3,3,16,6,Newark Liberty International,"Newark, NJ",DTW,"Detroit, MI",1400-1459,488,...,0.000124,4,Dallas Fort Worth Regional,0.00,0.0,0.0,56.0,17.67,0.0,0
4,4,6,20,4,Douglas Municipal,"Charlotte, NC",JFK,"New York, NY",1500-1559,541,...,0.000177,21,Myrtle Beach International,0.19,0.0,0.0,89.0,9.17,79.0,1


In [8]:

train_data.drop("Id", axis=1, inplace=True)

In [9]:

train_data.describe()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DISTANCE,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,MANUFACTURE_YEAR,NUMBER_OF_SEATS,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,...,FLT_ATTENDANTS_PER_PASS,GROUND_SERV_PER_PASS,PLANE_AGE,PRCP,SNOW,SNWD,TMAX,AWND,DEP_DELAY_NEW,IS_DELAYED
count,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.00000,...,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000,697224.000000
mean,6.620198,15.723732,3.931059,852.680886,3.064137,28.073635,2007.433931,134.799211,12398.966083,63379.64382,...,0.000099,0.000135,11.566069,0.104860,0.032057,0.090429,71.472508,8.369713,15.409382,0.399027
std,3.397212,8.756372,1.996984,611.715805,1.759975,21.329959,7.046577,47.035304,8315.616727,33670.96123,...,0.000085,0.000047,7.046577,0.342738,0.318871,0.724490,18.356162,3.609616,47.385170,0.489699
min,1.000000,1.000000,1.000000,66.000000,1.000000,1.000000,1987.000000,44.000000,1100.000000,835.00000,...,0.000000,0.000007,0.000000,0.000000,0.000000,0.000000,-10.000000,0.000000,0.000000,0.000000
25%,4.000000,8.000000,2.000000,402.000000,2.000000,11.000000,2002.000000,90.000000,5445.000000,25367.00000,...,0.000034,0.000099,5.000000,0.000000,0.000000,0.000000,59.000000,5.820000,0.000000,0.000000
50%,7.000000,16.000000,4.000000,687.000000,3.000000,23.000000,2007.000000,143.000000,11398.000000,68293.00000,...,0.000062,0.000125,12.000000,0.000000,0.000000,0.000000,74.000000,7.830000,0.000000,0.000000
75%,10.000000,23.000000,6.000000,1087.000000,4.000000,39.000000,2014.000000,173.000000,17499.000000,84667.00000,...,0.000144,0.000177,17.000000,0.020000,0.000000,0.000000,86.000000,10.510000,10.000000,1.000000
max,12.000000,31.000000,7.000000,5095.000000,15.000000,109.000000,2019.000000,337.000000,33340.000000,116615.00000,...,0.000348,0.000229,32.000000,11.630000,17.200000,25.200000,115.000000,33.780000,1742.000000,1.000000


In [10]:

pd.value_counts(train_data["IS_DELAYED"])

/var/folders/l2/m4x78b3x2pn811_9kxtgfwz80000gn/T/ipykernel_81016/1927735134.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(train_data["IS_DELAYED"])


IS_DELAYED
0    419013
1    278211
Name: count, dtype: int64

In [11]:
train_data.columns

Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'DEPARTING_AIRPORT',
       'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 'DEP_TIME_BLK',
       'DISTANCE', 'SEGMENT_NUMBER', 'CONCURRENT_FLIGHTS', 'MANUFACTURE_YEAR',
       'NUMBER_OF_SEATS', 'CARRIER_NAME', 'AIRPORT_FLIGHTS_MONTH',
       'AIRLINE_FLIGHTS_MONTH', 'AIRLINE_AIRPORT_FLIGHTS_MONTH',
       'AVG_MONTHLY_PASS_AIRPORT', 'AVG_MONTHLY_PASS_AIRLINE',
       'CARGO_HANDLING', 'FLT_ATTENDANTS_PER_PASS', 'GROUND_SERV_PER_PASS',
       'PLANE_AGE', 'PREVIOUS_AIRPORT', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND',
       'DEP_DELAY_NEW', 'IS_DELAYED'],
      dtype='object')

In [12]:
# redundant features
train_data.drop(['ORIGIN_CITY_NAME', 'DEST_CITY_NAME', 'MANUFACTURE_YEAR', 'DEP_DELAY_NEW',], axis=1, inplace=True)

In [13]:
train_data.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEPARTING_AIRPORT,DEST,DEP_TIME_BLK,DISTANCE,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,...,FLT_ATTENDANTS_PER_PASS,GROUND_SERV_PER_PASS,PLANE_AGE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND,IS_DELAYED
0,1,17,4,Raleigh-Durham International,JFK,1800-1859,427,6,13,76,...,0.000000,0.000094,5,Ronald Reagan Washington National,0.01,0.0,0.0,49.0,4.70,1
1,2,27,3,San Jose International,BOS,2200-2259,2689,2,3,162,...,0.000160,0.000127,17,John F. Kennedy International,0.21,0.0,0.0,64.0,12.75,0
2,8,14,3,Los Angeles International,ATL,0600-0659,1947,1,30,199,...,0.000144,0.000149,23,NONE,0.00,0.0,0.0,74.0,8.50,0
3,3,16,6,Newark Liberty International,DTW,1400-1459,488,3,23,76,...,0.000001,0.000124,4,Dallas Fort Worth Regional,0.00,0.0,0.0,56.0,17.67,0
4,6,20,4,Douglas Municipal,JFK,1500-1559,541,4,24,128,...,0.000098,0.000177,21,Myrtle Beach International,0.19,0.0,0.0,89.0,9.17,1


In [14]:

from sklearn.preprocessing import LabelEncoder

dest_le = LabelEncoder().fit(train_data["DEST"].tolist() + ["UNSEEN"])
train_data["DEST"] = dest_le.transform(train_data["DEST"])

carrier_name_le = LabelEncoder().fit(train_data["CARRIER_NAME"].tolist()+ ["UNSEEN"])
train_data["CARRIER_NAME"] = carrier_name_le.transform(train_data["CARRIER_NAME"])

previous_airport_le = LabelEncoder().fit(train_data["PREVIOUS_AIRPORT"].tolist()+ ["UNSEEN"])
train_data["PREVIOUS_AIRPORT"] = previous_airport_le.transform(train_data["PREVIOUS_AIRPORT"])

dep_time_blk_le = LabelEncoder().fit(train_data["DEP_TIME_BLK"].tolist()+ ["UNSEEN"])
train_data["DEP_TIME_BLK"] = dep_time_blk_le.transform(train_data["DEP_TIME_BLK"])

departing_airport_le = LabelEncoder().fit(train_data["DEPARTING_AIRPORT"].tolist()+ ["UNSEEN"])
train_data["DEPARTING_AIRPORT"] = departing_airport_le.transform(train_data["DEPARTING_AIRPORT"])


In [15]:
train_data = train_data[["DEPARTING_AIRPORT", "NUMBER_OF_SEATS", "PRCP", "CARRIER_NAME", "AIRLINE_FLIGHTS_MONTH", "DEP_TIME_BLK", "AIRPORT_FLIGHTS_MONTH", "MONTH", "DAY_OF_MONTH","DAY_OF_WEEK", "DEST", "DISTANCE", "PLANE_AGE", "SNOW", "SNWD", "TMAX", "AWND", "IS_DELAYED"]]


In [16]:
train_data.head()

,DEPARTING_AIRPORT,NUMBER_OF_SEATS,PRCP,CARRIER_NAME,AIRLINE_FLIGHTS_MONTH,DEP_TIME_BLK,AIRPORT_FLIGHTS_MONTH,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEST,DISTANCE,PLANE_AGE,SNOW,SNWD,TMAX,AWND,IS_DELAYED
0,67,76,0.01,6,19709,13,4567,1,17,4,172,427,5,0.0,0.0,49.0,4.70,1
1,77,162,0.21,9,22053,17,4249,2,27,3,44,2689,17,0.0,0.0,64.0,12.75,0
2,41,199,0.00,5,87457,1,19366,8,14,3,21,1947,23,0.0,0.0,74.0,8.50,0
3,51,76,0.00,11,26929,9,10379,3,16,6,98,488,4,0.0,0.0,56.0,17.67,0
4,17,128,0.19,1,76099,10,19017,6,20,4,172,541,21,0.0,0.0,89.0,9.17,1


In [17]:
train_data_np = train_data.to_numpy()

In [19]:
X = train_data_np[:, :-1] 
y = train_data_np[:, -1]

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [21]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [22]:
model = XGBClassifier()

In [23]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [24]:
from sklearn.metrics import roc_auc_score

In [25]:

y_test_predictions_probabilities = model.predict_proba(X_test)

In [26]:
y_test_predictions_probabilities

array([[0.7406115 , 0.2593885 ],
       [0.80061287, 0.19938713],
       [0.6771002 , 0.32289982],
       ...,
       [0.51945823, 0.48054177],
       [0.81530523, 0.18469477],
       [0.63200456, 0.36799544]], dtype=float32)

In [27]:
y_test_predictions = y_test_predictions_probabilities[:, 1]

In [28]:
roc_auc_score(y_test, y_test_predictions)

0.7147929247700309

In [29]:
test_data = pd.read_csv("data_test.csv")

In [30]:
test_data

,Id,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEPARTING_AIRPORT,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,DEP_TIME_BLK,DISTANCE,...,CARGO_HANDLING,FLT_ATTENDANTS_PER_PASS,GROUND_SERV_PER_PASS,PLANE_AGE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
0,0,7,1,1,Minneapolis-St Paul International,"Minneapolis, MN",PHL,"Philadelphia, PA",2000-2059,980,...,1462,0.000144,0.000149,18,Bradley International,0.00,0.0,0.0,93.0,4.70
1,1,4,12,5,Los Angeles International,"Los Angeles, CA",BNA,"Nashville, TN",1000-1059,1797,...,1462,0.000144,0.000149,18,Cincinnati/Northern Kentucky International,0.00,0.0,0.0,73.0,12.30
2,2,10,11,5,Ronald Reagan Washington National,"Washington, DC",SYR,"Syracuse, NY",0700-0759,298,...,0,0.000000,0.000090,16,Norfolk International,0.00,0.0,0.0,75.0,5.82
3,3,9,26,4,LaGuardia,"New York, NY",ORD,"Chicago, IL",2000-2059,733,...,518,0.000254,0.000229,12,Chicago O'Hare International,0.00,0.0,0.0,82.0,9.40
4,4,8,13,2,Detroit Metro Wayne County,"Detroit, MI",ERI,"Erie, PA",2000-2059,164,...,10,0.000034,0.000099,16,Kalamazoo/Battle Creek International,0.00,0.0,0.0,88.0,7.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298806,298806,6,16,7,Chicago O'Hare International,"Chicago, IL",AVL,"Asheville, NC",1800-1859,536,...,10,0.000034,0.000099,17,Duluth International,0.07,0.0,0.0,60.0,9.62
298807,298807,3,20,3,Tampa International,"Tampa, FL",EWR,"Newark, NJ",0700-0759,997,...,518,0.000254,0.000229,4,NONE,0.00,0.0,0.0,76.0,8.50
298808,298808,10,6,7,John F. Kennedy International,"New York, NY",MCO,"Orlando, FL",2100-2159,944,...,62,0.000160,0.000127,13,Orlando International,0.01,0.0,0.0,71.0,10.96
298809,298809,7,17,3,General Mitchell Field,"Milwaukee, WI",LGA,"New York, NY",0600-0659,738,...,10955,0.000062,0.000099,19,NONE,0.00,0.0,0.0,84.0,6.71


In [31]:
test_data = test_data[["Id", "DEPARTING_AIRPORT", "NUMBER_OF_SEATS", "PRCP", "CARRIER_NAME", "AIRLINE_FLIGHTS_MONTH", "DEP_TIME_BLK", "AIRPORT_FLIGHTS_MONTH", "MONTH", "DAY_OF_MONTH","DAY_OF_WEEK", "DEST", "DISTANCE", "PLANE_AGE", "SNOW", "SNWD", "TMAX", "AWND"]]

In [32]:
test_data.head()

,Id,DEPARTING_AIRPORT,NUMBER_OF_SEATS,PRCP,CARRIER_NAME,AIRLINE_FLIGHTS_MONTH,DEP_TIME_BLK,AIRPORT_FLIGHTS_MONTH,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEST,DISTANCE,PLANE_AGE,SNOW,SNWD,TMAX,AWND
0,0,Minneapolis-St Paul International,132,0.0,Delta Air Lines Inc.,86274,2000-2059,14787,7,1,1,PHL,980,18,0.0,0.0,93.0,4.70
1,1,Los Angeles International,160,0.0,Delta Air Lines Inc.,76299,1000-1059,17338,4,12,5,BNA,1797,18,0.0,0.0,73.0,12.30
2,2,Ronald Reagan Washington National,50,0.0,Comair Inc.,23411,0700-0759,11637,10,11,5,SYR,298,16,0.0,0.0,75.0,5.82
3,3,LaGuardia,120,0.0,United Air Lines Inc.,51182,2000-2059,13731,9,26,4,ORD,733,12,0.0,0.0,82.0,9.40
4,4,Detroit Metro Wayne County,50,0.0,SkyWest Airlines Inc.,70841,2000-2059,14439,8,13,2,ERI,164,16,0.0,0.0,88.0,7.61


In [33]:
new_dep_time_blk = []
for value in test_data["DEP_TIME_BLK"]:     
       if value not in dep_time_blk_le.classes_:
              new_dep_time_blk.append("UNSEEN")
       else:
              new_dep_time_blk.append(value)
# Replace
test_data["DEP_TIME_BLK"] = new_dep_time_blk

/var/folders/l2/m4x78b3x2pn811_9kxtgfwz80000gn/T/ipykernel_81016/3404122360.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["DEP_TIME_BLK"] = new_dep_time_blk


In [34]:
new_departing_airport = []
for value in test_data["DEPARTING_AIRPORT"]:
       if value not in departing_airport_le.classes_:
              new_departing_airport.append("UNSEEN")
       else:
              new_departing_airport.append(value)
# Replace
test_data["DEPARTING_AIRPORT"] = new_departing_airport

/var/folders/l2/m4x78b3x2pn811_9kxtgfwz80000gn/T/ipykernel_81016/3730070825.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["DEPARTING_AIRPORT"] = new_departing_airport


In [36]:
new_carrier = []
for value in test_data["CARRIER_NAME"]:
       if value not in carrier_name_le.classes_:
              new_carrier.append("UNSEEN")
       else:
              new_carrier.append(value)
# Replace
test_data["CARRIER_NAME"] = new_carrier



/var/folders/l2/m4x78b3x2pn811_9kxtgfwz80000gn/T/ipykernel_81016/3062908694.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["CARRIER_NAME"] = new_carrier


In [37]:
test_data

,Id,DEPARTING_AIRPORT,NUMBER_OF_SEATS,PRCP,CARRIER_NAME,AIRLINE_FLIGHTS_MONTH,DEP_TIME_BLK,AIRPORT_FLIGHTS_MONTH,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEST,DISTANCE,PLANE_AGE,SNOW,SNWD,TMAX,AWND
0,0,Minneapolis-St Paul International,132,0.00,Delta Air Lines Inc.,86274,2000-2059,14787,7,1,1,PHL,980,18,0.0,0.0,93.0,4.70
1,1,Los Angeles International,160,0.00,Delta Air Lines Inc.,76299,1000-1059,17338,4,12,5,BNA,1797,18,0.0,0.0,73.0,12.30
2,2,Ronald Reagan Washington National,50,0.00,Comair Inc.,23411,0700-0759,11637,10,11,5,SYR,298,16,0.0,0.0,75.0,5.82
3,3,LaGuardia,120,0.00,United Air Lines Inc.,51182,2000-2059,13731,9,26,4,ORD,733,12,0.0,0.0,82.0,9.40
4,4,Detroit Metro Wayne County,50,0.00,SkyWest Airlines Inc.,70841,2000-2059,14439,8,13,2,ERI,164,16,0.0,0.0,88.0,7.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298806,298806,Chicago O'Hare International,50,0.07,SkyWest Airlines Inc.,68083,1800-1859,26585,6,16,7,AVL,536,17,0.0,0.0,60.0,9.62
298807,298807,Tampa International,173,0.00,United Air Lines Inc.,52866,0700-0759,7388,3,20,3,EWR,997,4,0.0,0.0,76.0,8.50
298808,298808,John F. Kennedy International,162,0.01,JetBlue Airways,24795,2100-2159,10448,10,6,7,MCO,944,13,0.0,0.0,71.0,10.96
298809,298809,General Mitchell Field,143,0.00,Southwest Airlines Co.,116615,0600-0659,2559,7,17,3,LGA,738,19,0.0,0.0,84.0,6.71


In [38]:
test_data["DEP_TIME_BLK"] = dep_time_blk_le.transform(test_data["DEP_TIME_BLK"])
test_data["DEPARTING_AIRPORT"] = departing_airport_le.transform(test_data["DEPARTING_AIRPORT"])
test_data["CARRIER_NAME"] = carrier_name_le.transform(test_data["CARRIER_NAME"])
test_data["DEST"] = dest_le.transform(test_data["DEST"])

/var/folders/l2/m4x78b3x2pn811_9kxtgfwz80000gn/T/ipykernel_81016/4023277089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["DEP_TIME_BLK"] = dep_time_blk_le.transform(test_data["DEP_TIME_BLK"])
/var/folders/l2/m4x78b3x2pn811_9kxtgfwz80000gn/T/ipykernel_81016/4023277089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["DEPARTING_AIRPORT"] = departing_airport_le.transform(test_data["DEPARTING_AIRPORT"])
/var/folders/l2/m4x78b3x2pn811_9kxtgfwz80000gn/T/ipykernel_81016/4023277089.py:

In [39]:
test_data

,Id,DEPARTING_AIRPORT,NUMBER_OF_SEATS,PRCP,CARRIER_NAME,AIRLINE_FLIGHTS_MONTH,DEP_TIME_BLK,AIRPORT_FLIGHTS_MONTH,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEST,DISTANCE,PLANE_AGE,SNOW,SNWD,TMAX,AWND
0,0,48,132,0.00,5,86274,15,14787,7,1,1,251,980,18,0.0,0.0,93.0,4.70
1,1,41,160,0.00,5,76299,5,17338,4,12,5,42,1797,18,0.0,0.0,73.0,12.30
2,2,71,50,0.00,4,23411,2,11637,10,11,5,318,298,16,0.0,0.0,75.0,5.82
3,3,36,120,0.00,16,51182,15,13731,9,26,4,239,733,12,0.0,0.0,82.0,9.40
4,4,16,50,0.00,12,70841,15,14439,8,13,2,106,164,16,0.0,0.0,88.0,7.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298806,298806,10,50,0.07,12,68083,13,26585,6,16,7,25,536,17,0.0,0.0,60.0,9.62
298807,298807,84,173,0.00,16,52866,2,7388,3,20,3,111,997,4,0.0,0.0,76.0,8.50
298808,298808,31,162,0.01,9,24795,16,10448,10,6,7,203,944,13,0.0,0.0,71.0,10.96
298809,298809,22,143,0.00,13,116615,1,2559,7,17,3,190,738,19,0.0,0.0,84.0,6.71


In [40]:
test_data_np = test_data.to_numpy()

In [41]:
X_TEST = test_data_np[:, 1:] 

In [42]:
predictions = model.predict_proba(X_TEST)[:, 1] 

In [43]:
model = XGBClassifier()

In [44]:
predictions

array([0.47159535, 0.47780454, 0.23170175, ..., 0.43958828, 0.18798997,
       0.40605754], dtype=float32)

In [45]:
len(predictions)

298811

In [46]:
submission = test_data[["Id"]]

In [47]:
submission

,Id
0,0
1,1
2,2
3,3
4,4
...,...
298806,298806
298807,298807
298808,298808
298809,298809


In [48]:
submission["IS_DELAYED"] = predictions

/var/folders/l2/m4x78b3x2pn811_9kxtgfwz80000gn/T/ipykernel_81016/3329059061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission["IS_DELAYED"] = predictions


In [49]:
submission

,Id,IS_DELAYED
0,0,0.471595
1,1,0.477805
2,2,0.231702
3,3,0.470425
4,4,0.318245
...,...,...
298806,298806,0.694758
298807,298807,0.100235
298808,298808,0.439588
298809,298809,0.187990


In [44]:
submission.to_csv("test_submission.csv", index=False)